<a href="https://colab.research.google.com/github/iamatul1214/Tensorflow_Certification_Preparation/blob/main/06_Transfer_learning_using)tensorflow_part3_scaling_up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Till now we saw that feature extraction and Fine tuning transfer learning has worked nicely, now we will try to implement same on food101 dataset of 10% training data with 101 food classes.

In [1]:
import os
Root = "/content/drive/MyDrive/Tensorflow certifications work"
!pwd
os.chdir(Root)
!pwd

/content
/content/drive/MyDrive/Tensorflow certifications work


## installing the helper functions file

In [2]:
try:
  if os.path.exists("Helper.py"):
    print("Helper file already exists..skipping download")
  else:
    !wget https://raw.githubusercontent.com/iamatul1214/Tensorflow_Certification_Preparation/main/Helper.py
    print("downloaded Helper.py")
except Exception as e:
  print(e)

Helper file already exists..skipping download


In [3]:
## Downloading the dataset
from Helper import unzip_data
try:
  if os.path.exists("101_food_classes_10_percent"):
    print("data already exists...hence, skipping downloading....")
  else:
    !wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
    unzip_data("101_food_classes_10_percent.zip")
    print("unzipping the data")
except Exception as e:
  print(e)


--2022-10-29 11:46:46--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.11.128, 108.177.13.128, 74.125.26.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.11.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip’

101_food_classes_10 100%[===================>]   1.51G  68.6MB/s    in 23s     

2022-10-29 11:47:09 (68.2 MB/s) - ‘101_food_classes_10_percent.zip’ saved [1625420029/1625420029]

unzipping the data


In [5]:
## Setting up the data loaders
import tensorflow as tf
train_dir = "101_food_classes_10_percent/train"
test_dir = "101_food_classes_10_percent/test"
IMG_SIZE = (224,224)

full_classes_10_percent_train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                                         label_mode = 'categorical',
                                                                                         image_size = IMG_SIZE)

test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode = "categorical",
                                                                image_size = IMG_SIZE,
                                                                shuffle = False)

Found 7575 files belonging to 101 classes.
Found 25250 files belonging to 101 classes.


## train a big dog model with transfer learning on 10% of the 101 food classes

Here are the steps we will follow-
* Create a ModelCheckpoint callback
* Create a data augmentation layer to build a data augmentation right into the moddel
* BUild a headless(no top layers) functional EfficientNetB0 backboned-model ( we will create our own output layer) 
* compile our model
* feature extract for 5 full passes( 5 epochs on the train dataset and validate on 15% of the test data, to save epoch time)

In [6]:
## creating a checkpoint callback
from Helper import create_tensorboard_callback
checkpoint_path = "101_classes_10_percent_data_model_checkpoint"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                         monitor = 'val_accuracy',
                                                         save_best_only = True,
                                                         save_weights_only = True)


In [19]:
## Data augmentation layer to incorporate right into the model
from tensorflow.keras.layers.experimental import preprocessing

## setup data augmentation
data_augmentation = tf.keras.Sequential([
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2),
    preprocessing.RandomZoom(0.2),
    # preprocessing.Rescaling(1/255.). ## not required for efficientNetB0 but for resnet50
], name = "Data_augmentataiton_layer")
